In [3]:
import osmnx as ox
from osmnx import settings, graph, geocoder
import time
settings.use_cache = True

In [2]:
country = 'IN'

In [5]:
print(geocoder.geocode(query=country))

(40.3270127, -86.1746933)


In [2]:
cust_filter = '["route"~"bus"]'

In [3]:
def download_public_transport_networks(city_names):
    """Downloads public transport networks for a list of cities."""

    for city_name in city_names:
        try:
            city_name = str(city_name)
            print(f"Downloading network for {city_name}...")
            G = ox.graph_from_place(city_name, network_type='all',custom_filter=cust_filter)
            ox.save_graph_geopackage(G,filename=f"{city_name}_public_transport_network")
            print(f"Network for {city_name} downloaded successfully.")

            # Consider rate limiting to avoid overwhelming servers
            time.sleep(0.5)  # Adjust delay as needed

        except Exception as e:
            print(f"Error occurred while downloading network for {city_name}: {e}")

def get_cities_from_osm(num_cities=5):
    """Retrieves city names from OSM within a specified bounding box,
    leveraging place boundary geometries for accuracy.
    """

    bbox=[90,-90,180,-180]  # Global bounding box (adjust if needed)

    try:
        # Retrieve place geometries for more accurate city identification
        places_gdf = ox.features_from_bbox(bbox[0],bbox[1],bbox[2],bbox[3], tags={'place': 'city'})
        
        # cities_gdf = places_gdf.sort_values('population', ascending=False).head(num_cities)
        print(places_gdf)
        # city_names = list(cities_gdf['name'])
        
        return places_gdf

    except Exception as e:
        print(f"Error occurred while retrieving cities: {e}")
        return []


In [1]:
# city_names = get_cities_from_osm(num_cities=5)

In [2]:
# Download public transport networks
# download_public_transport_networks(city_names)